In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

import sys
import numpy as np

from PySDM.backends import CPU
from PySDM.builder import Builder
from PySDM.environments import Box
from PySDM.dynamics import Collision, Breakup, Coalescence
from PySDM.initialisation.sampling.spectral_sampling import ConstantMultiplicity

from PySDM.products.size_spectral import ParticlesVolumeSpectrum, ParticleConcentration, MeanRadius
from PySDM.products.collision.collision_rates import CollisionRatePerGridbox, CollisionRateDeficitPerGridbox, CoalescenceRatePerGridbox, BreakupRatePerGridbox
from PySDM.products.housekeeping.timers import WallTime

from matplotlib import pyplot

from PySDM.initialisation.spectra import Exponential
from PySDM.dynamics.collisions.kernels import Golovin, Geometric, ConstantK
from PySDM.physics.coalescence_efficiencies import Berry1967, ConstEc, Schlottke2010
from PySDM.physics.breakup_efficiencies import ConstEb
from PySDM.physics.breakup_fragmentations import AlwaysN, SLAMS, ExponFrag, Gaussian
from PySDM.physics.constants import si
from PySDM.formulae import Formulae
from pystrict import strict

from numba import config, threading_layer
config.THREADING_LAYER = 'default'

ImportError: cannot import name 'ParticlesVolumeSpectrum' from 'PySDM.products.size_spectral' (/Users/Emily/Documents/PySDM/PySDM/products/size_spectral/__init__.py)

In [5]:
@strict
class Settings:

    def __init__(self):
        self.formulae = Formulae()
        self.n_sd = 2**2
        self.n_part = 100 / si.cm**3
        self.X0 = self.formulae.trivia.volume(radius=30.531 * si.micrometres)
        self.dv = 1 * si.cm**3
        self.norm_factor = self.n_part * self.dv
        self.rho = 1000 * si.kilogram / si.metre**3
        self.dt = 1 * si.seconds
        self.adaptive = False
        self.seed = 44
        self._steps = [0, 1]
        #self.kernel = Golovin(b=1500 / si.second)
        self.kernel = ConstantK(a = 100.0 * si.cm**3 / si.second)
        self.coal_eff = ConstEc(Ec=1.0)
        self.fragmentation = Gaussian(mu=10 * si.micrometres, scale=5 * si.micrometres) #ExponFrag(scale=1e-6) #AlwaysN(n=1)
        self.break_eff = ConstEb(1.0) # no "bouncing"
        self.spectrum = Exponential(norm_factor=self.norm_factor, scale=self.X0)
        self.radius_bins_edges = np.logspace(np.log10(1 * si.um), np.log10(100 * si.um), num=128, endpoint=True)
        self.radius_range = [0 * si.um, 1e6 * si.um]

    @property
    def output_steps(self):
        return [int(step/self.dt) for step in self._steps]

In [12]:
settings = Settings()
backend = CPU

builder = Builder(n_sd=settings.n_sd, backend=backend(settings.formulae))
builder.set_environment(Box(dv=settings.dv, dt=settings.dt))
attributes = {}
attributes['volume'], attributes['n'] = ConstantMultiplicity(settings.spectrum).sample(settings.n_sd)
#breakup = Collision(settings.kernel, settings.coal_eff, settings.break_eff, settings.fragmentation, adaptive=settings.adaptive)
breakup = Breakup(settings.kernel, settings.fragmentation, adaptive=settings.adaptive)
builder.add_dynamic(breakup)
products = [ParticlesVolumeSpectrum(radius_bins_edges=settings.radius_bins_edges, name='dv/dlnr'),
            CollisionRatePerGridbox(name='cr'),CollisionRateDeficitPerGridbox(name='crd')]#,
            #BreakupRatePerGridbox(name='br'),CoalescenceRatePerGridbox(name='cor')]
core = builder.build(attributes, products)

for step in settings.output_steps:
    core.run(step - core.n_steps)
    pyplot.step(x=settings.radius_bins_edges[:-1] / si.micrometres, 
                y=core.products['dv/dlnr'].get()[0] * settings.rho,
                where='post', label="t = {step*settings.dt}s")
    #print(core.products['cr'].get(), core.products['crd'].get())
    #print(core.products['cor'].get(), core.products['br'].get())
    
pyplot.xscale("log")
pyplot.xlabel("radius (um)")
pyplot.ylabel("dm/dlnr")
pyplot.legend([0, 1, 2])

KeyError: 'Collision'